In [ ]:
- title: testing testing
- author: Jesus Caro
- date: 2023-11-19
- category: python
- tags: docker, vscode, github, codespaces, development containers, poetry, venv
- Subcells: [2,None]

Testing does it show this cell?

How about this cell?

And this cell?